In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

client = Elasticsearch('localhost:9200')

In [ ]:
s = Search(using=client, index="dataset") \
    .filter("term", category="search") \
    .query("match", title="python")   \
    .exclude("match", description="beta")

In [4]:
s = Search(using=client, index="dataset").query("match", text="DVD")   

In [5]:
response = s.execute()
response

<Response: [<Hit(dataset/12): {'Unnamed: 0': 12, 'page': 12, 'title': '映画 ドラマ  マーケット', 'te...}>, <Hit(dataset/11): {'Unnamed: 0': 11, 'page': 11, 'title': 'エン タメ  マーケット', 'tex...}>, <Hit(dataset/3): {'Unnamed: 0': 3, 'page': 3, 'title': ' 買い物  マーケット', 'text':...}>, <Hit(dataset/6): {'Unnamed: 0': 6, 'page': 6, 'title': '習う  マーケット', 'text': '...}>, <Hit(dataset/2): {'Unnamed: 0': 2, 'page': 2, 'title': '音楽  マーケット', 'text': '...}>, <Hit(dataset/893): {'Unnamed: 0': 893, 'page': 893, 'title': '乗り物  便利 赤ちゃん 旅行  ...}>]>

In [6]:
for hit in response:
    print(hit.meta.score, hit.title)

8.678354 映画 ドラマ  マーケット
8.516943 エン タメ  マーケット
7.8591595  買い物  マーケット
7.08802 習う  マーケット
6.9799786 音楽  マーケット
5.2839513 乗り物  便利 赤ちゃん 旅行  持ち物 リスト  トラベル


In [10]:
import json

In [32]:
search_param = {
    "query": {
        "match": {
            "text": "DVD",
#             "fuzziness": "AUTO",
#             "fuzzy_transpositions": True,
#             "operator":  "or",
#             "minimum_should_match": 1,
#             "analyzer": "my_kuromoji_analyzer"
        }
    }
}

In [43]:
# give high importance to specific fileds, here 4 times than text
search_param = {
  "query": {
    "multi_match": {
      "query": "DVD",
      "fields": ["title^4", "text"]
    }
  }
}

In [49]:
## SELECT * FROM table in SQL.
{ "match_all": {} }
{ "match": { "description": "Fourier analysis signals processing" }}
{ "match": { "date": "2014-09-01" }}
{ "match": { "visible": true }}

## all documents with the tag “math”. 
## The second query finds all documents with the tags “math” or “statistics”.
{ "term": { "tag": "math" }}
{ "terms": { "tag": ["math", "statistics"] }}

## run across multiple fields instead of just one.
{
  "multi_match": {
    "query": "probability theory",
    "fields": ["title", "body"]
  }
}

{
    "bool": {
        "must":     { "term": { "tag":    "math" }},
        "must_not": { "term": { "tag":    "probability"  }},
        "should": [
                    { "term": { "favorite": true   }},
                    { "term": { "unread":  true   }}
        ]
    }
}

{
   "filtered": {
     "query": { "match": { "body": "Probability Theory" }},
     "filter": { 
        "bool": {
          "must": {
            "range":  { "upvotes" : { "gt" : 20 } } 
           },
          "must_not": { "term":  { "tag": "frequentist" } }
        }
     }
  }
}

In [50]:
res= client.search(index='dataset', body=search_param)
for hit in res['hits']['hits']:
    print(hit['_source']['text'])
    print(hit['_score'])
    print('**********************')

In [ ]:
res= es.search(index='megacorp',body={
        'query':{
            'bool':{
                'must':[{
                        'match':{
                            'first_name':'nitin'
                        }
                    }]
            }
        }
    })

In [ ]:
res= es.search(index='megacorp',body={
        'query':{
            'bool':{
                'must':{
                    'match':{
                        'first_name':'nitin'
                    }
                },
                "filter":{
                    "range":{
                        "age":{
                            "gt":25
                        }
                    }
                }
            }
        }
    })
print res['hits']['hits']

In [22]:
res= client.search(index='dataset', body={
        'query':{
            'match':{
                "text":"検索 時間 再度"
            }
        }
    })
for hit in res['hits']['hits']:
    print(hit['_source']['text'])
    print(hit['_score'])
    print('**********************')

 マガジン 資格 スキル アップ  ショッピング 雑誌 本 初回 日間 無料  ためし 各種 設定  前  ただいま 検索 結果  取得 でき ませ  時間  改めて 再度 検索   ください  指定  検索 条件  該当 商品  あり ませ  でし  あなた  おすすめ ビジネス ランキング  検索 結果 検索 キーワード  入力   ください もしかして 検索 ビジネス 使え  使う あなた  合っ  コンテンツ  表示   ます 閉じる
16.442574
**********************
初回 日間 無料  ためし  ミュージック 音楽 洋楽  POP  ヒッツ  POP 前  各種 設定  ただいま 検索 結果  取得 でき ませ  時間  改めて 再度 検索   ください  指定  検索 条件  該当 商品  あり ませ  でし  アニメ 使え  使う あなた  合っ  コンテンツ  表示   ます 芸能 アイドル ランキング  検索 結果 閉じる 検索 キーワード  入力   ください もしかして 芸能 アイドル 検索 あなた  おすすめ  ショッピング テレビ
16.442574
**********************
CD DVD 音楽  ショッピング 洋楽  POP  ミュージック  POP  ヒッツ アニメ 初回 日間 無料  ためし 各種 設定  前  ただいま 検索 結果  取得 でき ませ  時間  改めて 再度 検索   ください  指定  検索 条件  該当 商品  あり ませ  でし  ヒップ ホップ   レゲエ 使え  使う あなた  合っ  コンテンツ  表示   ます あなた  おすすめ 音楽 ランキング  検索 結果 閉じる 検索 キーワード  入力   ください もしかして 検索 雑誌 本
16.290785
**********************
スポーツ ウェア 用品 雑誌 本 趣味 アウトドア スポーツ  マガジン 初回 日間 無料  ためし  ショッピング  ブック スポーツ  ゲーム ただいま 検索 結果  取得 でき ませ  時間  改めて 再度 検索   ください 動画 ゲーム 各種 設定  前   指定  検索 条件  該当 商品  あり ませ  でし   アニメ ストア あなた  

In [17]:
response = client.search(index="dataset", body=search_param)

In [ ]:
print(response['hits']['hits'])

In [ ]:
response

In [ ]:
## Phrase Search
## match exact sequence of words of phrases.
res= es.search(index='megacorp',doc_type='employee',body={
        'query':{
            'match_phrase':{
                "about":"play cricket"
            }
        }
    })